<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer: 
A neural network can be thought of as a function. The input layer contains the inputs to that function/network.
### Hidden Layer:
A hidden layer takes an input from the previous layer, puts it through a set of functions (weights for each input, a bias value, and a sigmoid or other normalization function), gets an output, and sends it to the next layer.
### Output Layer:
The output layer is the final layer in the neural network, i.e. the function's output.
### Neuron:
A neuron takes inputs from all neurons in the previous layer, calculates an output, and sends that output to the following layer.
### Weight:
Each input for a neuron is multiplied by a weight. Higher weights indicate greater importance. 
### Activation Function:
Activation functions determine how much signal is passed on to the subsequent layer. For instance, a sigmoid activation function will normalize output to be between 0 and 1.
### Node Map:
A node map shows the basic structure of a neural network.
### Perceptron:
A perceptron is the most basic neural network: one input layer and an output layer containing a single neuron.

## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

The inputs go into the input layer (each input a different neuron). Every single input is then sent to all neurons in the following layer. Each of those neurons is a function that contains weights, a bias, and an activation function.

Neuron's function: inputs are multiplied by its weight -> add bias -> insert into activation function -> output values are sent to the next layer.

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [2]:
import pandas as pd
import numpy as np
data = { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')

In [115]:
X = df[['x1', 'x2']]
y = df['y']

In [116]:
def perceptron_nand(X):

    weights = np.ones(len(X.columns))
    bias = -1
    weighted_sum = np.dot(X, weights) + bias
    
    def activate(x):
        return np.piecewise(x, [x <= 0, x > 0], [1, 0])
    
    return activate(weighted_sum)

In [117]:
perceptron_nand(X)

array([1., 1., 1., 0.])

## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [3]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [4]:
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.model_selection import train_test_split

feats = list(diabetes)[:-1]
target = list(diabetes)[-1:]

# Instantiate normalizer
normalizer = Normalizer()

# Normalize the feature data
X = normalizer.fit_transform(diabetes[feats])
# Set target vector
y = diabetes[target]

# Split data in train, test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Unit Step Activation Function

In [16]:
class Perceptron:

    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.activation_func = self._unit_step_func
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # init parameters
        self.weights = np.zeros(n_features)
        self.bias = 0

        y_ = np.array(y)

        for _ in range(self.n_iters):
            
            for idx, x_i in enumerate(X):

                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.activation_func(linear_output)
                
                # Perceptron update rule
                update = self.lr * (y_[idx] - y_predicted)

                self.weights += update * x_i
                self.bias += update

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = self.activation_func(linear_output)
        return y_predicted

    def _unit_step_func(self, x):
        return np.where(x>=0, 1, 0)

In [20]:
from sklearn.metrics import accuracy_score

p = Perceptron(learning_rate=0.01, n_iters=1000)
p.fit(X_train, y_train)

predictions = p.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print("Perceptron classification accuracy:", "\t", '{:.2f}%'.format(accuracy*100))

Perceptron classification accuracy: 	 74.03%


### Sigmoid Activation Function (not working properly)

In [168]:
from scipy.special import expit, logit

# Create perceptron object
class Perceptron(object):
    
    def __init__(self, n_iter = 10):
        self.n_iter = n_iter
    
    def __sigmoid(self, x):
        """ return sigmoid of a given value """
        return expit(x)
    
    def __sigmoid_derivative(self, x):
        """ return the sigmoid derivative of a given value """
        sx = self.__sigmoid(x)
        return sx * (1 - sx)

    def fit(self, X, y):
        """Fit training data
        X : Training vectors, X.shape : [#samples, #features]
        y : Target values, y.shape : [#samples]
        """

        # Randomly Initialize Weights
        weights = 2 * np.random.random((X.shape[1], 1)) - 1
        
        for i in range(self.n_iter):
            # Weighted sum of inputs / weights
            weighted_sum = np.dot(X, weights)
            # Activate!
            activated_output = self.__sigmoid(weighted_sum)
            # Cac error
            error = y - activated_output
            
            adjustments = error * self.__sigmoid_derivative(activated_output)
            
            # Update the Weights
            weights += np.dot(X.T, adjustments)
        return weights

    def predict(self, X):
        """Return class label after unit step"""
        weighted_sum = np.dot(X, weights)
        activated_output = self.__sigmoid(weighted_sum)
        return [1 if n > 0.5 else 0 for n in activated_output]

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?